In [ ]:
import digitalhub as dh
PROJECT_NAME = "deforestation"
proj = dh.get_or_create_project(PROJECT_NAME)

Register to the open data space copernicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [ ]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 2

In [ ]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2",
     "relativeOrbitNumber": "022"
 },
 "startDate": "2019-07-15",
 "endDate": "2019-07-30",
 "geometry": "POLYGON((11.90670 46.43071, 11.90670 46.43271, 11.90870 46.43271, 11.90870 46.43071, 11.90670 46.43071))",
 "area_sampling": "true",
 "cloudCover": "[0,5]",
 "artifact_name": "data_s2_deforestation" 
 }"""

list_args =  ["main.py",string_dict_data]


In [ ]:
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.14.6",command="python")

In [ ]:
run = function_s2.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    resources={"cpu": {"requests": "3", "limits": "6"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-deforestation",
        "mount_path": "/app/files",
        "spec": {
             "size": "350Gi" 
        }
    }])

### Log shape artifact

Log the shape file 'bosco' which can be downloaded from the [WebGIS Portal](https://webgis.provincia.tn.it/) confine del bosco layer or from https://siatservices.provincia.tn.it/idt/vector/p_TN_3d0874bc-7b9e-4c95-b885-0f7c610b08fa.zip. Unzip the files in a folder named 'bosco' and then log it

In [ ]:
artifact_name='bosco'
src_path='bosco'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

### Data elaboration

Note: For TQS and TPR tile type , please use version ghcr.io/tn-aixpa/rs-deforestation:3.1_TQS_TPR_B1 
For TPS tile type use the version below¶

In [ ]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-deforestation:0.14.6", command="/bin/bash", code_src="launch.sh")

### Pipeline

In [ ]:
%%writefile "deforestation_pipeline.py"

from hera.workflows import Workflow, DAG, Parameter
from digitalhub_runtime_hera.dsl import step

def pipeline():
    # Create a new Workflow with an entrypoint DAG and a parameter
    with Workflow(entrypoint="dag", arguments=[
        Parameter(name="geometry"),
        Parameter(name="outputName"),
        Parameter(name="startYear"),
        Parameter(name="endYear"),
        Parameter(name="shapeArtifactName"),
        Parameter(name="dataArtifactName")
        ]) as w:
        
        with DAG(name="dag"):
            string_dict_data = """{"satelliteParams":{"satelliteType":"Sentinel2",  "relativeOrbitNumber": "022"},"startDate":\""""+ str(w.get_parameter("startYear")) + """-01-01\","endDate": \"""" + str(w.get_parameter("endYear")) + """-12-31\","geometry": \"""" + str(w.get_parameter("geometry")) + """\","area_sampling":"true","cloudCover":"[0,5]","artifact_name": \"""" + str(w.get_parameter("dataArtifactName")) + """\"}"""          
             
            s1 = step(template={"action":"job",
                           "args":["main.py", string_dict_data], 
                           "secrets":["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"], 
                           "fs_group":"8877", 
                           "resources":{"mem": "32Gi", "cpu": "6"}, 
                           "envs":[{"name": "TMPDIR", "value": "/app/files"}], 
                           "volumes":[{"volume_type": "persistent_volume_claim","name": "volume-deforestation","mount_path": "/app/files","spec": { "size": "300Gi" }}]}, 
                 function="download_images_s2", 
                 name="download"
                 )
             
            s2 = step(template={"action":"job",
                           "args": ['/shared/launch.sh', str(w.get_parameter("shapeArtifactName")), str(w.get_parameter("dataArtifactName")), "[" + str(w.get_parameter("startYear")) + ',' + str(w.get_parameter("endYear")) + "]", str(w.get_parameter("outputName"))],
                           "fs_group":"8877",
                           "resources":{"cpu": "6","mem":"64Gi"},
                           "envs":[{"name": "TMPDIR", "value": "/app/data"}],
                           "volumes":[{"volume_type": "persistent_volume_claim", "name": "volume-deforestation","mount_path": "/app/data","spec": { "size": "250Gi" }}]},
                 function="elaborate",
                 name="elaborate"
                 )
            
            s1 >> s2
             
    return w    


In [ ]:
workflow = proj.new_workflow(
name="pipeline_deforestation_gitversion",
kind="hera",
code_src="git+https://<username>:<personal_access_token>@github.com/tn-aixpa/rs-deforestation",
handler="src.deforestation_pipeline:pipeline")

*(Optional) In case of forseen changes, uncomment the following cell, one can create the workflow locally after modifying the code as shown in cell above.*

In [ ]:
workflow = proj.new_workflow(name="pipeline_deforestation", kind="hera", code_src= "deforestation_pipeline.py", handler = "pipeline")

In [ ]:
workflow.run(action="build", wait=True)

Create workflow using project repo source file

Run workflow (TPS tiles)

Note: _For TPS tile , please use version ghcr.io/tn-aixpa/rs-deforestation:0.14.6_

In [ ]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((10.968432350469937 46.093829019481056,10.968432350469937 46.09650743619973, 10.97504139531014 46.09650743619973,10.97504139531014 46.093829019481056, 10.968432350469937 46.093829019481056))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_2018_19_tps",
    "outputName": "deforestation_2018_19_tps"
    })

Run workflow (TPR tiles)

Note: _For TPR and TQS tile , please use version ghcr.io/tn-aixpa/rs-deforestation:3.1_TQS_TPR_B1_

In [ ]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((11.04350 45.87743, 11.04350 45.87943, 11.04550 45.87943, 11.04550 45.87743, 11.04350 45.87743))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_2018_19_tpr",
    "outputName": "deforestation_2018_19_tpr"
    })

Run workflow (TQS tiles)

In [ ]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((11.90670 46.43071, 11.90670 46.43271, 11.90870 46.43271, 11.90870 46.43071, 11.90670 46.43071))",
    "shapeArtifactName":"bosco",
    'dataArtifactName': "data_s2_2018_19_tqs",
    "outputName": "deforestation_2018_19_tqs"
    })

Note: One can use online service like wkt_geojson indicated below that convert geometry(wkt) to  GeoJSON which is then displayed on the map

https://9revolution9.com/tools/geo/wkt_geojson/